In [1]:
%pip install mosestokenizer

  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49170 sha256=a993029671d11c92f8173ddaefba40175364ef4c4d0fd38e25d6b1113bf94bf9
  Stored in directory: /root/.cache/pip/wheels/80/d8/15/4c5ebbe883513f003cb055a0369c77c9df857023a706f39e70
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3338 sha256=5e9b423ad35bc7745dae528221411b8a74f529ee8d784f1aa66ef9df001a4700
  Stored in directory: /root/.cache/pip/wheels/e1/af/b1/99b57a06dda78fdcee86d2e22c64743f3b8df8f31cfc04baf7
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6147 sha256=adfdb1243dd0342ef90cea5886889224a16be3f78b295b590804b2b60ad209a4
  Stored in directory: /root/.cache/pip/wheels/05/ee/10/33257b0801ac6a912c841939032c16da1eb3db377afe1443e5
Successfully built mosestokenizer toolwrapper uctools
Note: you may 

In [2]:
!pwd

/kaggle/working


In [3]:
HTML = "https://www.gutenberg.org/cache/epub/2814/pg2814-images.html"

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
dubliners = requests.get(HTML)

In [6]:
assert dubliners.status_code == 200

In [7]:
soup = BeautifulSoup(dubliners.text, 'html.parser')

In [8]:
body = soup.find("body")

In [9]:
text = []
for chapter in body.findAll("div", {"class": "chapter"}):
    for element in chapter.findChildren():
        if element.name == "section" and element.get("id") in ["pg-header", "pg-footer"]:
            continue
        stripped = element.text.strip()
        if stripped != "":
            text.append(stripped)

In [10]:
from mosestokenizer import MosesSentenceSplitter

In [11]:
sents = []
try:
    with MosesSentenceSplitter('en') as splitsents:
        for para in text:
          if para == "":
              continue
          sents += splitsents([para.replace("\r\n", " ")])
except Exception as ex:
    print(ex, para)

In [12]:
import re
def cleaner(text):
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("’ ", " ")
    text = text.replace("’", "'")
    text = text.replace("‘", " ")
    text = text.replace("\t", " ")
    text = text.replace("!...", " ")
    text = text.replace("....", " ")
    text = text.replace("...", " ")
    text = text.replace(":", " ")
    text = text.replace(";", " ")
    text = text.replace("!", " ")
    text = text.replace(",", " ")
    text = text.replace("?", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("—", " ")
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.replace(".", " ")
    text = text.replace("&", " and ")
    text = text.replace(" 57E ", " fifty seven e ")
    text = text.replace(" 1st ", " first ")
    text = text.replace(" 6th ", " sixth ")
    text = text.replace(" 1895 ", " eighteen ninety five ")
    text = text.replace(" 1891", " eighteen ninety one")
    text = text.replace("1891", " eighteen ninety one")
    text = text.replace("65 ", " sixty five ")
    if text[-1] == ".":
        text = text[:-1]
    text = re.sub("  +", " ", text)
    return text.lower().strip()

In [13]:
clean = [cleaner(x) for x in sents]

In [14]:
with open("dubliners-clean.txt", "w") as outf:
    for line in clean:
        outf.write(line + "\n")

In [15]:
!apt install -y build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev




build-essential is already the newest version (12.8ubuntu1.1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu1.5).
The following additional packages will be installed:
  autoconf automake autotools-dev bzip2-doc cpp-8 file gcc-8 gcc-8-base
  gfortran gfortran-8 gfortran-9 ibverbs-providers icu-devtools
  libboost-atomic-dev libboost-atomic1.71-dev libboost-atomic1.71.0
  libboost-chrono-dev libboost-chrono1.71-dev libboost-chrono1.71.0
  libboost-container-dev libboost-container1.71-dev libboost-container1.71.0
  libboost-context-dev libboost-context1.71-dev libboost-context1.71.0
  libboost-coroutine-dev libboost-coroutine1.71-dev libboost-coroutine1.71.0
  libboost-date-time-dev libboost-date-time1.71-dev libboost-date-time1.71.0
  libboost-exception-dev libboost-exception1.71-dev libboost-fiber-dev
  libboost-fiber1.71-dev libboost-fiber1.71.0 libboost-graph-dev
  libboost-graph-parallel-dev libboost-

In [16]:
%cd /tmp

/tmp


In [17]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 14161, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 14161 (delta 162), reused 406 (delta 131), pack-reused 13687
Receiving objects: 100% (14161/14161), 5.91 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (8042/8042), done.


In [18]:
%cd kenlm

/tmp/kenlm


In [19]:
!mkdir build
%cd build
!cmake ..
!make -j 4

/tmp/kenlm/build
cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by cmake)
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found suitable version "1.71.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Check if compiler accepts -pthread
-- 

In [20]:
%cd /kaggle/working

/kaggle/working


In [21]:
!/tmp/kenlm/build/bin/lmplz -o 5 < dubliners-clean.txt > dubliners.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /kaggle/working/dubliners-clean.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 68170 types 7521
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:90252 2:2627887616 3:4927289344 4:7883662848 5:11497009152
Statistics:
1 7521 D1=0.613663 D2=1.27413 D3+=1.35107
2 36763 D1=0.80197 D2=1.15443 D3+=1.52879
3 56953 D1=0.9043 D2=1.32639 D3+=1.55938
4 60067 D1=0.964639 D2=1.51858 D3+=1.79285
5 56906 D1=0.97755 D2=1.72265 D3+=1.46156
Memory estimate for binary LM:
type      kB
probing 4795 assuming -p 1.5
probing 5725 assuming -r models -p 1.5
trie    2209 without quantization
trie    1174 assuming -q 8 -b 8 quantization 
trie    2042 assuming -a 22 array pointer compression
trie    1007 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/